In [1]:
# The following code implements the modern McEliece cryptosystem in Sage. Note that SageMath 10 is required.

m = 7
t = 5
k.<a> = FiniteField(2^m)
R.<z> = PolynomialRing(k)


def chunks(lst, n):
# Yield successive n-sized chunks from lst.
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def translate_to_field(msg,m):
# take a message msg of m bits and translate it to a field element of F_2^m
    count = 0
    x = k.zero()
    for j in range(m):
        if msg[m-j-1]:
            x = x + a^j
    return x

def translate_to_field_vector(v,m):
# take a message vector v of m*t bits and translate it to a vector of t field elements of F_2^m
    result = []
    for j in chunks(v,m):
        result.append(translate_to_field(j,m))
    return result

def translate_from_field(x,m):
# take a field element x in 2^m and translate it to a message of m bits
    if (x == 0):
        return [0]*m
    if (x == 1):
        result = [0]*m
        result[0] = 1
        return result
    return x.list()

def translate_from_field_vector(x,m):
    vec = [translate_from_field(x[j],m) for j in range(len(x))]
    return matrix(vec).transpose()

def translate_from_field_matrix(M,m):
    # if matrix is t x n, the result is mt x n
    result = translate_from_field_vector(M[0,:].list(),m)
    for j in range(M.nrows()-1):
        result = result.stack(translate_from_field_vector(M[j+1,:].list(),m))
    return result

def generate_error(n,w):
# generates a (somewhat random) n-bit vector of Hamming weight exactly w
#    p = w/n
#    result = [0]*n
#    count = 0
#    while count == 0:
#        for j in range(n):
#            r = random()
#            if (r < p):
#                result[j] = 1
#                count = count + 1
#            if (count == w):
#                break
#   return result
    if (w > n):
        return vector(matrix.ones(1,n))
    result = [0]*(n-w)
    result.extend([1]*w)
    shuffle(result)
    return result
           
def trunc_poly(p,n):
    q = z;
    for j in range(0,n):
        p = (p-p(0)) // q
    return p

def parity_check_entry(g,i,j, list):
    # need 0<i<=t, 0<j<=n
    # outputs the (i,j)-entry of the parity check matrix of the Goppa code corresponding to g
    return (trunc_poly(g,t-i)(list[j]))/g(list[j])


In [2]:
class McEliece:

    def randomgoppa(self):
    # creates a random irreducible Goppa polynomial of degree t over F_2^m
        g = R.zero()
        while not (g.is_irreducible()):
            g = R.random_element(degree = t);
        return g;
        
    def keygen(self):
        g = self.randomgoppa()
        # generate a random permutation of all field elements (n = 2^m) and write it over F_2
        list_field = list(k)
        perm = Permutations(2^m).random_element()
        list_field_perm = [list_field[i-1] for i in perm]
        parity = matrix(k,t,2^m, lambda x,y: parity_check_entry(g,x,y,list_field_perm))
        parity = translate_from_field_matrix(parity,m)
        parity = parity.rref() 
        # want parity to be of the form (I_mt | Q) where Q is mt x (n-mt)
        # this doesn't necessarily happen immediately
        # swap columns, swapping the variables in the permutation as well
        new_parity = copy(parity)
        for row in range(m*t):
            if (parity[row,row] == 0):
                vec = vector(parity[row,:].list())
                for col in range(2^m):
                    if (vec[col] == 1):
                        new_parity.swap_columns(row, col)
                        list_field_perm[row], list_field_perm[col] = list_field_perm[col], list_field_perm[row]
                        break
        new_parity = new_parity.rref()
        Q = new_parity[:,m*t:2^m] # generator and parity check matrix can be deduced from this and don't need to be stored
        return (Q, g, list_field_perm)
    
    def __init__(self):
        (self.public_key, self.private_poly, self.private_perm) = self.keygen()

# note that a generates the finite field F_2^m, z is the indeterminate of the polynomial ring over this field, and x is the indeterminate of the quotient of this ring over the Goppa poly

    def encrypt(self, msg):
        # msg must have n-m*t bits
        G = self.public_key.stack(matrix.identity(2^m-m*t)) # generator matrix, n x (n-mt)
        error = vector(generate_error(2^m,t))
        msg = vector(GF(2), msg)
        # G*msg is in the Goppa code. Now add the random error and output to get the codeword
        return (G*msg) + error
    
    def EEA_patterson(self, r,g):
    # applies standard EEA, but terminates as soon as the degrees fulfill the conditions of Patterson's algorithm
        r_fixed = r
        g_fixed = g
        cap = g.degree()
        u = R.zero()
        u1 = R.one()
        v = R.one()
        v1 = R.zero()
        while r.degree() > floor(cap/2) or u1.degree() > floor((cap-1)/2):
            (quo,rem) = r.quo_rem(g)
            r = g; g = rem
            u2 = u1; u1 = u; u = u2-quo*u
            v2 = v1; v1 = v; v = v2-quo*v
            # r = u1*r_fixed + v1*g_fixed
        return (r,u1)

    def errorlocator(self, c):
        Q.<x> = R.quotient(self.private_poly(z))     
        # Apply Patterson's algorithm to derive the error locator polynomial over F_2^m
        # c = Gm + e   
        # compute syndrome s
        s = Q.zero()
        for j in range(2^m):
            s += c[j] / (x - self.private_perm[j])
        if (s == Q.zero()):
            return Q.zero()
        T = 1/s
        if (T == x):
            return x   
        # now find square root r of T(z)+z. Q is a field with 2^(mt) elements, so Frobenius is the identity
        r = (T+x)^(2^(m*t-1))
        r = R(r.list())
        (A,B) = self.EEA_patterson(r,self.private_poly)
        return (A^2+z*B^2)

    def error_from_errorlocator(self, sigma):
        # finds the roots of the error locator polynomial and turns it into an error vector, applying the secret permutation on the way
        error = [0]*2^m
        for j in range(2^m):
            if (sigma(self.private_perm[j]) == R.zero()):
                error[j] = 1
        return error
    
    def decrypt(self, c):
        sigma = self.errorlocator(c)
        error = vector(self.error_from_errorlocator(sigma))
        c = c-error # = Gm
        # now can read off msg as the last n-mt digits
        return c[(m*t):2^m]


        

In [3]:
# McEliece is initialised with values m,t. A message must have exactly 2^m-m*t bits, and the codeword will have 2^m bits.

# generate random msg with 2^m-m*t bits
msg = [0]*(2^m-m*t)
for i in range(2^m-m*t):
    r = random()
    if r>0.5:
        msg[i] = 1
print("This is the message")
print(msg)
cryptosystem = McEliece()
c = cryptosystem.encrypt(msg)
print("This is the codeword:")
print(c)
print("This is the decrypted codeword:")
print(cryptosystem.decrypt(c))


This is the message
[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
This is the codeword:
(0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1)
This is the decrypted codeword:
(1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,